In [20]:
import pyvisa
from pylabnet.utils.logging.logger import LogClient
from pylabnet.network.core.generic_server import GenericServer
from pylabnet.hardware.power_meter.thorlabs_pm320e import Driver
from pylabnet.hardware.polarization.polarization_control import MPC320, paddle2
import time

In [2]:
gpib_addres = 'USB0::0x1313::0x8022::M00579698::INSTR'

# Instantiate
logger = LogClient(
    host='139.180.129.96',
    port=12087,
    module_tag='Polarization Optimizer'
)

In [3]:
power_meter = Driver(
    gpib_address=gpib_addres, 
    logger=logger
)

In [50]:
power_meter.set_range(2, 'R100UW')

In [4]:
pol_paddle = MPC320()

There are 1 devices connected
Device list created succesfully
Connected to device 38154354,
Description:  b'Polarization Controller'
Serial No:  b'38154354'
Motor Type:  6553856
USB PID:  2397516216
Max Number of Paddles:  1
Connected succesfully to device


In [5]:
# Output of pol. paddle is connected to output 2 of powermeter
channel = 2
init_power = power_meter.get_power(channel)

In [10]:
print(f"Initial power is {init_power*1e6:.2f} uW.")

Initial power is 6.80 uW.


In [51]:
# Changing Polarization
paddle = paddle2


maxtravel = pol_paddle._polarizationdll.MPC_GetMaxTravel(pol_paddle.device_info.serialNo)
initialpos = maxtravel/4-1

home = pol_paddle._polarizationdll.MPC_Home(pol_paddle.device_info.serialNo, paddle)
pol_paddle.PosI = pol_paddle._polarizationdll.MPC_GetPosition(pol_paddle.device_info.serialNo, paddle)

print(f"Position before move is {pol_paddle.PosI}")
init_power = power_meter.get_power(channel)
print(f"Initial power is {init_power*1e6:.2f} uW.")

move = pol_paddle._polarizationdll.MPC_MoveToPosition(pol_paddle.device_info.serialNo, paddle, initialpos) 
time.sleep(10)
pol_paddle.PosIn = pol_paddle._polarizationdll.MPC_GetPosition(pol_paddle.device_info.serialNo, paddle)


print(f"Moved to final position: {pol_paddle.PosIn}") 
final_power = power_meter.get_power(channel)
print(f"Power after move is {final_power*1e6:.2f} uW.")

Position before move is 45.54014598540146
Initial power is 3.65 uW.
Moved to final position: 85.0
Power after move is 1.35 uW.
